Building a Normal RAG System with GROQ Inferencing - Loading the data from website (betterhealth) that deals with suicide and mental health conditions.

In [57]:
from langchain.document_loaders import WebBaseLoader # Initializing web base loader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # For chunking
from langchain_groq import ChatGroq # Inference
from langchain.chains.summarize import load_summarize_chain 
from langchain.chains.question_answering import load_qa_chain
import os # Detecting environment
from dotenv import load_dotenv

In [23]:
load_dotenv() # Loading the environment

True

In [24]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") # Loading the API Key

In [11]:
bthealthdocs = WebBaseLoader(web_path="https://www.betterhealth.vic.gov.au/health/conditionsandtreatments/suicide-and-mental-illness")

In [17]:
healthdocsloader = bthealthdocs.load() # Loading the document

In [18]:
healthdocsloader

[Document(page_content="\n\n\nSuicide and mental health conditions - Better Health Channel\n\n\n Skip to main content   Better Health Channel   \n        Our websites\n        View the list          Conditions and treatments  Conditions AllergiesAnxietyArthritisAsthmaBehavioural conditionsBirth defectsCancerCOVID-19DementiaDepressionDiabetesInfectionsMental health and wellbeingSexually transmissible infectionsSleepStrokePain and pain management Body parts Mouth and teethBlood and blood vesselsHair and nailsGenetic conditionsBones muscles and jointsEar nose and throatBody systems Fertility and the reproductive system - maleFertility and the reproductive system - femaleDigestive systemImmune systemHormonal system (endocrine)Neuromuscular system Organs Brain and nervesEyesLungsKidney and bladderHeartSkinTests and treatments Complementary and alternative careFirst AidSurgeryMedical testsMedications      Healthy living  Keeping healthy Health checksHealthy eatingHealthy mindKeeping activeWe

The text documents are extracted from the website

In [14]:
chunkdocs = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=0) # Performing chunking

In [20]:
splitdocs = chunkdocs.split_documents(healthdocsloader) # Splitting the documents

### Summarization - Better Health Website

Instancing LLM

In [45]:
groqllm = ChatGroq(model="llama3-8b-8192",temperature=0) # Llama3 model, 0 temp

In [46]:
chain = load_summarize_chain(groqllm,chain_type="map_reduce") # Chaining

In [47]:
chain.run(splitdocs)

'Here is a concise summary:\n\nThe article discusses the importance of recognizing the warning signs of suicide and seeking help. It highlights the link between mental health conditions, such as depression and anxiety, and suicidal thoughts and behaviors. The article encourages individuals to prioritize their mental well-being, seek support from loved ones and professionals, and recognize that seeking help is a sign of strength, not weakness. It also provides resources and support services for individuals struggling with suicidal thoughts or behaviors, as well as for those who want to help someone who is suicidal.'

Q&A

In [54]:
query = "Did australia affected because of suicide"

In [55]:
qachain = load_qa_chain(groqllm,chain_type="stuff") # laoding the qa chain

In [56]:
qachain.run(input_documents=splitdocs,question=query)

'According to the text, yes, Australia has been affected by suicide. The text states that "Over 3,000 Australians die from suicide each year, with mental health conditions being a significant risk factor." This is more than double the road toll. Additionally, the text mentions that men, people living in rural and remote regions, and Aboriginal and Torres Strait Islander people have a higher rate of suicide.'

It gave the response related to the query. Hitting the better health website and extracting data and then converting them to chunks, performing Q&A.